# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# load data from database
engine = create_engine('sqlite:///DisasterMessages.db')
df = pd.read_sql_table(
    'DisasterMessages',
    con=engine)
#df.info()
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Tokenize text
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('muc', MultiOutputClassifier(DecisionTreeClassifier(random_state=0)))
    #('muc', MultiOutputClassifier(KNeighborsClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

#pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)
df_test = pd.DataFrame(y_pred,columns=y_test.columns)
result_list = []


In [ ]:
for column in y_test.columns:
    print(classification_report(y_test[column], df_test[column], target_names = ['No', 'Yes']))
    results = classification_report(y_test[column], df_test[column], target_names = ['No', 'Yes'])
    result_list.append(results)
    

In [ ]:
#result_list

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
# show parameters
#pipeline.get_params().keys()

# specify parameters for grid search
parameters = {
    #'muc__estimator__n_neighbors': [100,150]
    'muc__estimator__min_samples_leaf': [40,100]
#'muc__estimator__max_depth': [1,5] #10,15,20
#'muc__estimator__n_estimators': [50, 100, 200],
#'muc__estimator__min_samples_split': [2,5,10]
}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs =-1)

In [ ]:
cv.fit(X_train, y_train)

# show best parameters
print("\nBest Parameters:", cv.best_params_)

In [8]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'muc', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'muc__estimator__class_weight', 'muc__estimator__criterion', 'muc__estimator__max_depth', 'muc__estimator__max_features', 'muc__estimator__max_leaf_nodes', 'muc__estimator__min_impurity_decrease', 'muc__estimator__min_impurity_split', 'muc__estimator__min_samples_leaf', 'muc__estimator__min_samples_split', 'muc__estimator__min_weight_fraction_leaf', 'muc__estimator__presort', 'muc__estimator__random_state', 'muc__estimator__splitter', 'muc__estimator', 'muc__n_jobs'])

In [ ]:
# show best parameters
print("\nBest Parameters:", cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', RandomForestClassifier())
    ])

    return pipeline

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
with open('final_model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.